In [1]:
%load_ext lab_black

In [2]:
import pandas as pd

In [3]:
def normalize_group(cms):
    if cms in {"eSchoolView", "Linq Websites"}:
        return "Linq Websites (formerly eSchoolView)"
    return cms

In [4]:
RAW_DATA_FILEPATH = "../data/what_cms_data_market_share.csv"
INDY_DATA_FILEPATH = "../data/indy_data.csv"
PUBLIC_FILEPATH = "../data/public_data.csv"

In [5]:
INDY_CUTOFF = 6
PUBLIC_CUTOFF = 15

In [6]:
COLUMNS = ["WHOCMS", "WHOCMS Previous 2021", "WHOCMS Previous 2020"]

In [7]:
EXCLUDE_INDUSTRIES = {"Higher Education", "International School"}

In [8]:
CMS = {
    "Finalsite",
    "SchoolPointe",
    "SOCS",
    "Blackboard",
    "SchoolMessenger",
    "Edlio",
    "Apptegy",
    "Gabbart or North American Division",
    "Educational Networks",
    "eSchoolView",
    "School Blocks",
    "Campus Suite",
    "School Loop",
    "Catapult CMS",
    "Blackbaud OnMessage",
    "Blackbaud",
    "School Webmasters",
    "eChalk",
    "SCHOOLinSITES",
    "Sitefinity",
    "eCatholic",
    "CivicEngage",
    "Linq Websites",
    "Revize",
    "elcomCMS",
    "Percussion",
    "OU Campus",
    "Moodle",
    "Aeries",
    "CMS4Schools",
    "Cnyric",
    "Foxbright",
    "Joomla",
    "Drupal",
}

In [9]:
pd.read_csv(RAW_DATA_FILEPATH, encoding="ISO-8859-1")

,Account Full ID,Account Name,Industry,Industry Normalized,Market Map,WHOCMS,WHOCMS Previous 2021,WHOCMS Previous 2020,WHOCMS Changed Date,WHOCMS Last Update Date,Client/Prospect,Current/Previous Client,Billing State/Province,Billing Country,Change Since Previous Run,CMS Size
0,0015a00002enSqhAAE,Sherman Thomas STEM Academy,Charter School,Charter School,Charter,1&amp;1 IONOS Website Builder,1&amp;1 IONOS Website Builder,NaN,NaN,8/22/21,Prospect,0,CA,United States,False,32.0
1,0013300001pbLVJAA2,Emmaus School,Independent School,International School,Tier 3,1&amp;1 IONOS Website Builder,1&amp;1 IONOS Website Builder,NaN,NaN,8/22/21,Prospect,0,Wiltshire,United Kingdom,False,32.0
2,0014000000zOKaQAAW,St John's Episcopal School,Independent School,Independent School,Tier 3,1&amp;1 IONOS Website Builder,1&amp;1 IONOS Website Builder,1&amp;1 IONOS Website Builder,7/16/20,8/22/21,Prospect,0,FL,United States,True,32.0
3,0014000000zOJ8PAAW,Childrens Own School,Montessori,Independent School,Tier 3,1&amp;1 IONOS Website Builder,1&amp;1 IONOS Website Builder,1&amp;1 IONOS Website Builder,7/16/20,8/22/21,Prospect,0,MA,United States,True,32.0
4,0014000000zOKShAAO,Jessie R Wagner Adventist Elementary,Independent School,Independent School,Tier 3,1&amp;1 IONOS Website Builder,1&amp;1 IONOS Website Builder,1&amp;1 IONOS Website Builder,7/16/20,8/22/21,Prospect,0,PA,United States,True,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43394,0015a00002eofJtAAI,Montgomery Christian Academy,Independent School,Independent School,Unknown,NaN,NaN,NaN,NaN,8/22/21,Prospect,0,TX,United States,False,NaN
43395,0015a00002encWhAAI,Star Academy for the Gifted and Talented,Independent School,Independent School,Unknown,NaN,NaN,NaN,NaN,8/22/21,Prospect,0,MA,United States,False,NaN
43396,0015a00002nfD51AAE,Diocese of Cleveland,Diocese,Independent School,Unknown,NaN,NaN,NaN,NaN,8/22/21,Prospect,0,OH,United States,False,NaN
43397,0014000000GsdfbAAB,St. Pius V School,Independent School,Independent School,Unknown,NaN,NaN,NaN,NaN,8/22/21,Prospect,0,NaN,United States,False,NaN


In [10]:
df = (
    pd.read_csv(RAW_DATA_FILEPATH, encoding="ISO-8859-1")
    .loc[lambda df: ~df["Industry"].isin(EXCLUDE_INDUSTRIES), COLUMNS + ["Industry"],]
    .assign(
        cms_2020_07=lambda df: df["WHOCMS Previous 2020"].apply(normalize_group),
        cms_2020_11=lambda df: df["WHOCMS Previous 2021"].apply(normalize_group),
        cms_2021_08=lambda df: df["WHOCMS"].apply(normalize_group),
    )
    .drop(COLUMNS, axis=1)
    .dropna()
)

In [11]:
public_df = (
    df[df["Industry"] == "Public School District"]
    .drop("Industry", axis=1)
    .apply(lambda s: s.value_counts())
    .fillna(0)
    .reset_index()
    .loc[lambda df: df["index"].isin(CMS)]
    .loc[lambda df: df["cms_2021_08"] > PUBLIC_CUTOFF]
    .rename(columns={"index": "cms"})
)

In [12]:
indy_df = (
    df[df["Industry"] == "Independent School"]
    .drop("Industry", axis=1)
    .apply(lambda s: s.value_counts())
    .fillna(0)
    .reset_index()
    .loc[lambda df: df["index"].isin(CMS)]
    .loc[lambda df: df["cms_2021_08"] > INDY_CUTOFF]
    .rename(columns={"index": "cms"})
)

In [13]:
public_df.to_csv(PUBLIC_FILEPATH, index=False)
indy_df.to_csv(INDY_DATA_FILEPATH, index=False)